In [ ]:
import sys
from loguru import logger
if "google.colab" in sys.modules:
    logger.info("Colab Detected")
    import tensorflow as tf
    gpus = tf.config.list_physical_devices("GPU")
    if not gpus:
        raise RuntimeError("Nessuna GPU trovata.Controlla di aver selezionato il runtime giusto.")
    else:
        logger.info(f"Trovate {len(gpus)} GPU:\n{gpus}")
    ! git clone https://github.com/AtomicDonuts/Progetto_Computings.git
    %cd Progetto_Computings/
    ! pip install numpy astropy loguru plotly matplotlib pandas
    ! python3 fits_import/fits2csv.py
    sys.path.append("imports/")
    import custom_variables as custom_paths
else:
    logger.info("WaldoSchaeffer Detected")
    sys.path.append("../imports/")
    import custom_variables as custom_paths

In [ ]:
import pandas as pd
import numpy as np
from keras.layers import Dense, Input, Concatenate,Flatten
from keras.models import Model
from tensorflow.keras.utils import plot_model

In [ ]:
df = pd.read_csv(custom_paths.csv_path)

In [ ]:
flux_band = np.array(
    [[f"Flux_Band_{i}", f"Sqrt_TS_Band_{i}"] for i in range(8)]
)  # len(Flux_Band)=8
flux_band = np.dstack((df[flux_band[:, 0]].to_numpy(), df[flux_band[:, 1]].to_numpy()))
flux_hist = np.array(
    [[f"Flux_History_{i}", f"Sqrt_TS_History_{i}"] for i in range(14)]
)  # len(Flux_History)=15
flux_hist = np.dstack((df[flux_hist[:, 0]].to_numpy(), df[flux_hist[:, 1]].to_numpy()))
print(f"Flux_Band Size: {flux_band.shape}")
print(f"Flux_History Size: {flux_hist.shape}")

In [ ]:
inputs = Input(shape=flux_band.shape[1:])
hidden = Flatten()(inputs)
hidden = Dense(200, activation="relu")(hidden)
hidden = Dense(200, activation="relu")(hidden)

In [ ]:
inputs2 = Input(shape=flux_hist.shape[1:])
hidden2 = Flatten()(inputs2)
hidden2 = Dense(200, activation="relu")(hidden2)
hidden2 = Dense(200, activation="relu")(hidden2)

In [ ]:
concat = Concatenate()([hidden, hidden2])
final_hidden = Dense(200, activation="relu")(concat)
outputs = Dense(1, activation="sigmoid")(final_hidden)

In [ ]:
model = Model(inputs=[inputs,inputs2], outputs=outputs)
model.compile(loss="binary_crossentropy", optimizer="adam")

In [ ]:
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True)